In [ ]:
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer
from pathlib import Path
from transformers import Pipeline
import torch.nn.functional as F
import torch
import os

model_dir = "/huggingface/sentence_transformers"
model_name = "bge-small-zh-v1.5"

model_path = os.path.join(model_dir, model_name)
model_path

/mnt/llm/feiyue/miniconda3/envs/onnx/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/huggingface/sentence_transformers/bge-small-zh-v1.5'

In [ ]:
onnx_path = "./tmp/bge_onnx"
# load vanilla transformers and convert to onnx
model = ORTModelForFeatureExtraction.from_pretrained(model_path, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead
Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: BertModel *****
Using framework PyTorch: 2.2.2+cu121
Overriding 1 configuration item(s)
	- use_cache -> False


/tmp/tmpmtal4pd2/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f2ec2c42c70>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f2ec3318f30>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f2ec3319070>]
/tmp/tmpmtal4pd2/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f2ec21ad530>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f2ec21ae2f0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f2ec21ad2b0>]


('./tmp/bge_onnx/tokenizer_config.json',
 './tmp/bge_onnx/special_tokens_map.json',
 './tmp/bge_onnx/vocab.txt',
 './tmp/bge_onnx/added_tokens.json',
 './tmp/bge_onnx/tokenizer.json')

In [ ]:
from onnxruntime.transformers import optimizer

optimized_model = optimizer.optimize_model(
    "./tmp/bge_onnx/model.onnx",
    model_type="bert",
    num_heads=12,
    hidden_size=768,
    opt_level=99,
    use_gpu=True,
    only_onnxruntime=True,
)
optimized_model.convert_float_to_float16()

optimized_model.save_model_to_file("./tmp/bge_onnx/model_fp16.onnx")

./tmp/bge_onnx/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f301bf6f130>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f2eb6d044b0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f2ebc744070>]


2024-07-11 16:30:22.993250331 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 4 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2024-07-11 16:30:22.994253030 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-07-11 16:30:22.994262661 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2024-07-11 16:30:23.029039355 [W:onnxruntime:, inference_session.cc:1978 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTran

In [ ]:
!python -m onnxruntime.transformers.compare_bert_results --use_gpu  --baseline_model ./tmp/bge_onnx/model.onnx --optimized_model ./tmp/bge_onnx/model_fp16.onnx --batch_size 1 --sequence_length 128 --samples 500



./tmp/bge_onnx/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fd572501c70>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fd572501cb0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fd5725026f0>]
2024-07-11 16:30:32.291283074 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 10 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
./tmp/bge_onnx/model_fp16.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fd5725026f0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fd572501c70>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fd572501cb0>]
2024-07-11 16:30:33.122091241 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 8 Memcpy nodes are added to the graph 

In [ ]:
!python -m onnxruntime.transformers.bert_perf_test --model ./tmp/bge_onnx/model_fp16.onnx --batch_size 1 --sequence_length 512 --opt_level 99 --use_gpu --use_io_binding

test setting TestSetting(batch_size=1, sequence_length=512, test_cases=10, test_times=100, use_gpu=True, use_io_binding=True, provider=None, intra_op_num_threads=None, seed=3, verbose=False, log_severity=2, average_sequence_length=512, random_sequence_length=False)
Generating 10 samples for batch_size=1 sequence_length=512
./tmp/bge_onnx/model_fp16.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f7c92b3dcf0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f7c912a9d70>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f7c91390ef0>]
2024-07-11 16:31:05.582628583 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 8 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
Running test: model=model_fp16.onnx,graph_optimization_level=ENABLE_AL

In [ ]:
!python -m onnxruntime.transformers.bert_perf_test --model ./tmp/bge_onnx/model.onnx --batch_size 1 --sequence_length 512 --use_gpu --use_io_binding

test setting TestSetting(batch_size=1, sequence_length=512, test_cases=10, test_times=100, use_gpu=True, use_io_binding=True, provider=None, intra_op_num_threads=None, seed=3, verbose=False, log_severity=2, average_sequence_length=512, random_sequence_length=False)
Generating 10 samples for batch_size=1 sequence_length=512
./tmp/bge_onnx/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f0e48991ef0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f0e47bc33b0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f0e486f8930>]
2024-07-11 14:24:25.161926851 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 4 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
Running test: model=model.onnx,graph_optimization_level=ENABLE_ALL,intra_op

In [ ]:
!python -m onnxruntime.transformers.bert_perf_test --model ./tmp/bge_onnx/model_fp16.onnx --batch_size 1 --sequence_length 512 --use_gpu --use_io_binding

test setting TestSetting(batch_size=1, sequence_length=512, test_cases=10, test_times=100, use_gpu=True, use_io_binding=True, provider=None, intra_op_num_threads=None, seed=3, verbose=False, log_severity=2, average_sequence_length=512, random_sequence_length=False)
Generating 10 samples for batch_size=1 sequence_length=512
./tmp/bge_onnx/model_fp16.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fc834b9bf70>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fc833cf1170>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7fc833cf11b0>]
2024-07-11 16:32:34.436910944 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 8 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
Running test: model=model_fp16.onnx,graph_optimization_level=ENABLE_AL

In [ ]:
!python -m onnxruntime.transformers.profiler --model ./tmp/bge_onnx/model.onnx --batch_size 1 --sequence_length 128 --samples 1000 --dummy_inputs bert --use_gpu

Arguments Namespace(input=None, model='./tmp/bge_onnx/model.onnx', batch_size=1, sequence_length=128, past_sequence_length=1, global_length=1, samples=1000, threshold=0.01, thread_num=-1, input_ids_name=None, segment_ids_name=None, input_mask_name=None, dummy_inputs='bert', use_gpu=True, provider='cuda', basic_optimization=False, kernel_time_only=False, verbose=False)
./tmp/bge_onnx/model.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f997b9c8570>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f997b9c85b0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f997b9c85f0>]
loading profile output onnxruntime_profile__2024-07-11_14-12-19.json ...
No kernel record found!

Nodes in the original order:
----------------------------------------------------------------
Total(μs)	Time%	Acc %	Avg(μs)	Calls	Provider	Node
     14037	 0.56	 0.56	     4.7	 3000	CUDA    	/Shape
      9964	 0.40	 0.95	     3.3	 3000	CPU     	/Gather
     

In [ ]:
!python -m onnxruntime.transformers.profiler --model ./tmp/bge_onnx/model_fp16.onnx --batch_size 1 --sequence_length 512 --samples 1000 --dummy_inputs bert --use_gpu

Arguments Namespace(input=None, model='./tmp/bge_onnx/model_fp16.onnx', batch_size=1, sequence_length=128, past_sequence_length=1, global_length=1, samples=1000, threshold=0.01, thread_num=-1, input_ids_name=None, segment_ids_name=None, input_mask_name=None, dummy_inputs='bert', use_gpu=True, provider='cuda', basic_optimization=False, kernel_time_only=False, verbose=False)
./tmp/bge_onnx/model_fp16.onnx
False
[<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f5f2afbd2f0>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f5f2afbd330>, <onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x7f5f2afbd370>]
loading profile output onnxruntime_profile__2024-07-11_16-31-41.json ...
No kernel record found!

Nodes in the original order:
----------------------------------------------------------------
Total(μs)	Time%	Acc %	Avg(μs)	Calls	Provider	Node
     21055	 0.79	 0.79	     7.0	 3000	CUDA    	/embeddings/word_embeddings/Gather
     12300	 0.46	 1.26